
**The `ai_parse_document()` function invokes a state-of-the-art generative AI model from Databricks Foundation Model APIs to extract structured content from unstructured documents.**


https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_parse_document?language=Python

In [0]:
from pyspark.sql.functions import *

In [0]:
ai_parse_document??

https://docs.databricks.com/aws/en/sql/language-manual/data-types/variant-type

In [0]:
from pyspark.sql.functions import *

df = spark.read.format("binaryFile") \
  .load("/Volumes/datasciencebasics/youtube/gpt4all/gpt4all.pdf") \
  .withColumn(
    "parsed",
    expr("ai_parse_document(content)"))
display(df)

In [0]:
display(df)

In [0]:
%sql
WITH parsed_documents AS (
    SELECT
      path,
      ai_parse_document(
        content,
        map(
          'imageOutputPath', '/Volumes/datasciencebasics/youtube/gpt4all/',
          'descriptionElementTypes', '*'
        )
      ) AS parsed
    FROM READ_FILES('/Volumes/datasciencebasics/youtube/gpt4all/gpt4all.pdf', format => 'binaryFile')
  ),
  parsed_text AS (
    SELECT
      path,
      concat_ws(
        '\n\n',
        transform(
          try_cast(parsed:document:elements AS ARRAY<VARIANT>),
          element -> try_cast(element:content AS STRING)
        )
      ) AS text
    FROM parsed_documents
    WHERE try_cast(parsed:error_status AS STRING) IS NULL
  )
  SELECT
    path,
    text,
    ai_query(
      'databricks-meta-llama-3-3-70b-instruct',
      concat(
        'Extract all author names ',
        text
      ),
      returnType => 'STRING'
    ) AS structured_data
  FROM parsed_text
  WHERE text IS NOT NULL;


In [0]:
from pyspark.sql.functions import col, substring

df_short = _sqldf.withColumn(
    "text_first_100",
    substring(col("text"), 1, 100)
)
display(df_short.select("text_first_100"))